# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [84]:
# import libraries
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
import pickle
from pprint import pprint
import re
import sys
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sqlalchemy import create_engine
import nltk
import warnings
warnings.filterwarnings('ignore')

In [105]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///messages.db')
    df = pd.read_sql_table("messages", engine)  
    X = df['message'].values
    Y = df.drop(['id', 'message', 'original', 'genre'],axis=1)
    Y['related'] = Y['related'].map(lambda x: 1 if x == 2 else x)
    categories = Y.columns
    
    return X, Y

In [106]:
X, Y = load_data()

In [107]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [108]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    
    #normalize
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #tokenize
    tokens = word_tokenize(text)
    
    #stop_words
    my_stopwords=stopwords.words('english')
    tokens = [word for word in tokens if word not in my_stopwords]
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
def simple_model():
    pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
    ]) 
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
# train classifier
model = simple_model()
model.fit(X_train, y_train)
    
# predict on test data
y_pred = model.predict(X_test)


In [111]:
msg = ['NEIGHBORHOODS DESTROYED: We\'re getting a closer look at the damage caused by multiple fires in the North Bay. This is the scene at Range Avenue and Bicentennial Way in Santa Rosa.']
test_output = model.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'weather_related']


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [90]:
def score_report():
    y_pred_pd = pd.DataFrame(y_pred, columns = y_test.columns)
    for column in y_test.columns:
        print('------------------------------------------------------\n')
        print('FEATURE: {}\n'.format(column))
        print(classification_report(y_test[column],y_pred_pd[column]))

In [110]:
score_report()

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.61      0.50      0.55      1503
           1       0.86      0.90      0.88      5051

    accuracy                           0.81      6554
   macro avg       0.73      0.70      0.72      6554
weighted avg       0.80      0.81      0.81      6554

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5447
           1       0.80      0.46      0.58      1107

    accuracy                           0.89      6554
   macro avg       0.85      0.72      0.76      6554
weighted avg       0.88      0.89      0.88      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1     

### 6. Improve your model
Use grid search to find better parameters. 

In [113]:
#using the same pipeline, add parameters
parameters = {
    'clf__estimator__n_estimators': [20, 50]
}

cv = GridSearchCV(estimator=model, param_grid=parameters, cv=3, scoring='f1_weighted', verbose=3)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ..... clf__estimator__n_estimators=20, score=0.539, total=  51.3s
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   51.3s remaining:    0.0s


[CV] ..... clf__estimator__n_estimators=20, score=0.537, total=  59.6s
[CV] clf__estimator__n_estimators=20 .................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  1.8min remaining:    0.0s


[CV] ..... clf__estimator__n_estimators=20, score=0.532, total= 1.0min
[CV] clf__estimator__n_estimators=50 .................................
[CV] ..... clf__estimator__n_estimators=50, score=0.550, total= 2.8min
[CV] clf__estimator__n_estimators=50 .................................
[CV] ..... clf__estimator__n_estimators=50, score=0.552, total= 2.8min
[CV] clf__estimator__n_estimators=50 .................................
[CV] ..... clf__estimator__n_estimators=50, score=0.538, total= 2.9min


[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 11.4min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                            

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [114]:
y_pred = cv.predict(X_test)

In [115]:
score_report()

------------------------------------------------------

FEATURE: related

              precision    recall  f1-score   support

           0       0.69      0.46      0.55      1503
           1       0.85      0.94      0.89      5051

    accuracy                           0.83      6554
   macro avg       0.77      0.70      0.72      6554
weighted avg       0.82      0.83      0.82      6554

------------------------------------------------------

FEATURE: request

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      5447
           1       0.83      0.53      0.64      1107

    accuracy                           0.90      6554
   macro avg       0.87      0.75      0.79      6554
weighted avg       0.90      0.90      0.89      6554

------------------------------------------------------

FEATURE: offer

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6524
           1     

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6008
           1       0.92      0.26      0.41       546

    accuracy                           0.94      6554
   macro avg       0.93      0.63      0.69      6554
weighted avg       0.94      0.94      0.92      6554

------------------------------------------------------

FEATURE: storm

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      5946
           1       0.78      0.42      0.55       608

    accuracy                           0.94      6554
   macro avg       0.86      0.71      0.76      6554
weighted avg       0.93      0.94      0.93      6554

------------------------------------------------------

FEATURE: fire

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6493
           1       0.50      0.02      0.03        61

    accuracy                           

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(), n_jobs=-1))
])
parameters = {
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4]
} 
cv = GridSearchCV(pipeline, parameters)
cv.fit(X_train, y_train)

### 9. Export your model as a pickle file

In [ ]:
with open('classifer.pkl', 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.